# 01 Datenaufbereitung



In [ ]:
# ================================================================
# Beschreibung:     BTH 04 - Rekonstruktion Stadtmodell Basel 1960
# Erstellt mit:     Unterstützung durch ChatGPT (OpenAI)
# Version:          GPT-4, Juni 2025
# Autor:            Marco Stampfli und Vania Fernandes Pereira
# ================================================================



## Was passiert in diesem Schritt?

Wir bereiten die Daten so auf, dass wir die Punktwolke als Grundlage im `.txt`-Format erhalten.  
Die Punktwolke wurde aus **Leica 3Dr** exportiert und hat folgendes Format:

![Export aus Leica 3Dr](../../../docs/img/Export_PW_3Dr_bearbeitet.png)

*Bemerkung: Aus dem Export sollte XYZ auf 2 Kommastellen exportiert werden (nicht wie im Bild), jedoch die Normalen mit 6 Nachkommastellen. Dies ist später wichtig bei der Vergleichbarkeit verschiedener Punktwolken!!! Dazu wird noch ein Skript erstellt um dieses Problem zu beheben*

---

## Dateibenennung

Basis-Exportdateien sollen nach folgendem Schema gespeichert werden.  
Verwende dabei sinnvolle, sprechende Kurzformen:

- **PW** = Punktwolke  
- **P3A1** = Platte 3, Ausschnitt 1

In [ ]:
# Datenart_Kachel.txt

file_path = '/input/PW_P3A1.txt'


Die txt-Datei sollte so aus sehen:

| KOO     | RBG 0-255 | Normalen 0-1 |
| ------- | --------- | ------------ |
| X, Y, Z | R, G, B   | x, y, z      |
| ...     | ...       | ...          |

Der folgende Code kann in folgende Teilschritte gegliedert werden:
1. Benötigte Pakages laden
2. Einlesen der txt-Datei
3. Spalten zuweisen im eingelesenen DataFrame
4. Bearbeiten der Spalten und normieren der Werte RGB, HSV berechnen
5. Spalten neuanordnen [KOO, RGB, HSV, xyzNormalen]
6. Speichern des neuen DataFrames als txt

## CODE - Normalisieren 

Dieser Code wurde mit Hilfe von ChatGPT erstellt und kommt aus \PyCode\XX_high_speed_Normiert.py und wurde angepasst für das Github-Repo.

In [ ]:
import pandas as pd
import colorsys
import numpy as np
import time
import os

# ================================================================
# Beschreibung:     BTH 04 - Rekonstruktion Stadtmodell Basel 1960
# Erstellt mit:     Unterstützung durch ChatGPT (OpenAI)
# Version:          GPT-4, Juni 2025
# Autor:            Marco Stampfli und Vania Fernandes Pereira
# ================================================================

start_total = time.time()

#file_path = '/input/PW_P3.txt'
file_path = '/input/PW_P3A1.txt'  # ← ggf. anpassen
#file_path = '/input/PW_P3A2.txt'
#file_path = '/input/PW_P3A3.txt'
file_path = r"arbeitspakete\02_segmentierung\04_Segm_Dachformen\input\Diverse_komplexe_Dachtypen.txt"

# Dateinamen ohne Erweiterung und Erweiterung selbst extrahieren
filename = os.path.basename(file_path)  # z.B. 'PW_P3A1.txt'
name, ext = os.path.splitext(filename)  # name = 'PW_P3A1', ext = '.txt'

# Neuen Dateinamen mit "_normalisiert" erzeugen
new_filename = f"{name}_normalisiert{ext}"

# Ausgabepfad erzeugen
output_path = os.path.join('/output', new_filename)

# Datei einlesen (Semikolon-getrennt, ohne Header)
start_read = time.time()
df = pd.read_csv(file_path, delimiter=";", decimal=".", header=None)
end_read = time.time()

# Spaltennamen manuell zuweisen
df.columns = ["X coordinate", "Y coordinate", "Z coordinate",
              "Red color (0-255)", "Green color (0-255)", "Blue color (0-255)",
              "X scan dir", "Y scan dir", "Z scan dir", "Klasse"]

# RGB normalisieren (0–1)
start_rgb = time.time()
df["R_norm"] = df["Red color (0-255)"] / 255.0
df["G_norm"] = df["Green color (0-255)"] / 255.0
df["B_norm"] = df["Blue color (0-255)"] / 255.0
end_rgb = time.time()

# RGB → HSV (vektorisiert)
start_hsv = time.time()
rgb_norm = df[["R_norm", "G_norm", "B_norm"]].to_numpy()
hsv = np.apply_along_axis(lambda x: colorsys.rgb_to_hsv(*x), 1, rgb_norm)
df[["Hue_norm", "Saturation_norm", "Value_norm"]] = pd.DataFrame(hsv, index=df.index)
end_hsv = time.time()

# Spalten in gewünschter Reihenfolge anordnen
df = df[[
    "X coordinate", "Y coordinate", "Z coordinate",
    "R_norm", "G_norm", "B_norm",
    "Hue_norm", "Saturation_norm", "Value_norm",
    "X scan dir", "Y scan dir", "Z scan dir", "Klasse"
]]

# Datei speichern
start_save = time.time()
df.to_csv(output_path, sep=";", index=False, decimal=".", header=False)
end_save = time.time()

end_total = time.time()

# Zeit-Report
print(f"\n✅ Verarbeitung abgeschlossen.")
print(f"📥 Einlesen der Datei:       {end_read - start_read:.2f} Sek.")
print(f"🎨 RGB-Normalisierung:      {end_rgb - start_rgb:.2f} Sek.")
print(f"🌈 HSV-Umrechnung:          {end_hsv - start_hsv:.2f} Sek.")
print(f"💾 Speichern der Datei:     {end_save - start_save:.2f} Sek.")
print(f"⏱️ Gesamtlaufzeit:           {end_total - start_total:.2f} Sek.")


FileNotFoundError: [Errno 2] No such file or directory: 'arbeitspakete\\02_segmentierung\\04_Segm_Dachformen\\input\\Diverse_komplexe_Dachtypen.txt'

**PW_P3A1_normalisiert.txt**

Die txt-Datei sollte nun so aus sehen:

| KOO     | RBG 0-1 | HSV 0-1 | Normalen 0-1 |
| ------- | ------- | ------- | ------------ |
| X, Y, Z | R, G, B | H, S, V | x, y, z      |
| ...     | ...     | ...     | ...          |

## CODE - Normalisieren 2

Ordner einlesen und alle Dateien mit .txt Endung lesen und Normalisieren.


**Input:** PW_P3A1_Ground.txt

**Output:** PW_P3A1_Ground_normalisiert.txt


Dieser Code wurde mit Hilfe von ChatGPT erstellt und kommt aus den Dateien:

\PyCode\XX_high_speed_Normiert.py
\PyCode\000_nomalyze.py

und wurde angepasst für das Github-Repo.

In [ ]:
import pandas as pd
import numpy as np
import colorsys
from pathlib import Path
import os
import time

start_total = time.time()
# === EINSTELLUNGEN ===
folder_name = "/Klass_P3A1" # Ordnername im Inputordner
input_folder = Path(f"./input{folder_name}")  # Ordner mit den TXT-Dateien
output_folder = Path(f"./output{folder_name}_normalisiert")  # Zielordner für die normalisierten Dateien
output_folder.mkdir(exist_ok=True)

# === ALLE .txt-DATEIEN IM ORDNER VERARBEITEN ===
for input_file in input_folder.glob("*.txt"):
    # Datei einlesen (Semikolon-getrennt, ohne Header)
    start_read = time.time()
    df = pd.read_csv(input_file, delimiter=";", decimal=".", header=None)
    end_read = time.time()


    # Spaltennamen manuell zuweisen
    df.columns = ["X coordinate", "Y coordinate", "Z coordinate",
                "Red color (0-255)", "Green color (0-255)", "Blue color (0-255)",
                "X scan dir", "Y scan dir", "Z scan dir", "Klasse"]

    # RGB normalisieren (0–1)
    start_rgb = time.time()
    df["R_norm"] = df["Red color (0-255)"] / 255.0
    df["G_norm"] = df["Green color (0-255)"] / 255.0
    df["B_norm"] = df["Blue color (0-255)"] / 255.0
    end_rgb = time.time()

    # RGB → HSV (vektorisiert)
    start_hsv = time.time()
    rgb_norm = df[["R_norm", "G_norm", "B_norm"]].to_numpy()
    hsv = np.apply_along_axis(lambda x: colorsys.rgb_to_hsv(*x), 1, rgb_norm)
    df[["Hue_norm", "Saturation_norm", "Value_norm"]] = pd.DataFrame(hsv, index=df.index)
    end_hsv = time.time()

    # Spalten in gewünschter Reihenfolge anordnen
    df = df[[
        "X coordinate", "Y coordinate", "Z coordinate",
        "R_norm", "G_norm", "B_norm",
        "Hue_norm", "Saturation_norm", "Value_norm",
        "X scan dir", "Y scan dir", "Z scan dir", "Klasse"
    ]]

    # Datei speichern
    start_save = time.time()
    output_file = output_folder / f"{input_file.stem}_normalisiert.txt"
    df.to_csv(output_file, sep=";", index=False, header=False, decimal=".")

    print(f"✔ {input_file.name} → gespeichert als {output_file.name}")
    end_save = time.time()

end_total = time.time()

# Zeit-Report
print(f"\n✅ Verarbeitung abgeschlossen.")
print(f"📥 Einlesen der Datei:       {end_read - start_read:.2f} Sek.")
print(f"🎨 RGB-Normalisierung:      {end_rgb - start_rgb:.2f} Sek.")
print(f"🌈 HSV-Umrechnung:          {end_hsv - start_hsv:.2f} Sek.")
print(f"💾 Speichern der Datei:     {end_save - start_save:.2f} Sek.")
print(f"⏱️ Gesamtlaufzeit:           {end_total - start_total:.2f} Sek.")
